# Séparation de sources

In [ ]:
local = 1
if local== 0:
    import io
    import soundfile as sf
    from urllib.request import urlopen

    url1 = "https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Mix11.wav"
    url2 = "https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Mix21.wav"
    url3 = "https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Mix12.wav"
    url4 = "https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Mix22.wav"
    V1, samplerate = sf.read(io.BytesIO(urlopen(url1).read()))
    V2, samplerate = sf.read(io.BytesIO(urlopen(url2).read()))
    V3, samplerate = sf.read(io.BytesIO(urlopen(url3).read()))
    V4, samplerate = sf.read(io.BytesIO(urlopen(url4).read()))
    
    V1=2**14*V1
    V2=2**14*V2
    V3=2**14*V3
    V4=2**14*V4
else :
    V1="Mix11.wav"
    V2="Mix21.wav"
    V3="Mix12.wav"
    V4="Mix22.wav"

In [ ]:
from __future__ import division
from IPython.display import Audio
import numpy as np
import scipy as scp
import pylab as pyl
from matplotlib import cm
import matplotlib.pyplot as plt
import pywt
import scipy.io as sio
from scipy import fftpack
from matplotlib.pyplot import imshow as imageplot
from mpl_toolkits.mplot3d import Axes3D
import wave
import warnings
from scipy.io.wavfile import read
warnings.filterwarnings('ignore')
import math

Le but de ce travail est de réaliser un programme python qui effectue une séparation de sources. Plus précisément un programme qui prend en entrée un son stéréophonique, mélange instantané de plusieurs sources sonores (ici des instruments de musique) et qui renvoie plusieurs sons stéréophoniques. On utilisera la transformée de Fourier à court terme (TFCT), ou transformée de Fourier à fenêtres. 
    Le procédé général de ce programme de séparation de sources est intégralement décrit dans le poly de cours. Je vous invite à vous y reporter pour de plus amples explications. En bref, on va utiliser le fait, 
    que la plupart du temps, deux instrusments ne jouent pas la même note au même moment et on va segmenter les spectrogrammes des deux voix de manière à attribuer chaque atome temps-fréquence à un instrument, en fonction du rapport d'intensité entre les deux voix, qui est supposé caractérisé chaque instrument.

Le travail se décompose en trois parties.

*1) On considère un mélange de seulement deux instruments et on fait l'hypothèse que l'on connait les coefficients du mélange.*

*2) On considère un mélange de trois instruments et on fait l'hypothèse que l'on connait les coefficients du mélange.*

*3) On considère le même mélange de trois instruments mais cette fois ci on estime les coefficients de mélange.*

# **Séparation avec connaissance des coefficients de mélange**

Tout d'abord nous nous situons dans un cas où nous connaissons le nombre d’instruments ainsi que les coefficients de mélange des deux voix. Le but va alors être à partir des Transformées de Fourier à Court Terme (TFCT, c’est-à-dire le diagramme temps-fréquence du signal sonore) de créer les spectrogrammes associés à chaque instrument pour chaque voix. Pour cela, nous associons les coefficients caractéristiques d’un instrument aux deux spectrogrammes de cet instrument. Une fois cela effectué, nous reconstituons le son de chaque instrument sortant de chaque voix. 

Pour réaliser cela, nous nous appuierons sur le spectrogramme et la manière dont il propose une représentation parcimonieuse du signal sonore.

De plus, nous admettrons les propriétés suivantes : 
* Les instruments jouant au même moment ne jouent pas à la même fréquence (en considérant un grand nombre de fenêtres temporelles), et s'ils jouent à la même fréquence ne jouent pas au même moment.
* Il n'y a pas de retard d'un instrument par rapport aux autres sur les différentes voix : on parle de mélange simultané.

## Séparation de deux instruments

*A l'aide la fonction read, charger dans deux vecteurs différents $V1$ et $V2$ les sons Mix11.wav et Mix21.wav et afficher les tranformées de Fourier à fenêtres de chacune des deux voix (on utilisera TFCourtTerme).
On pourra afficher les basses fréquences pour mieux voir.
On fait l'hypothèse que les voix 1 et 2 (dites V1 et V2 ) est obtenue à partir des deux sources S1 et S2 de la manière suivante :
$$V_1=\frac{1}{3}S_1+\frac{2}{3}S_2\text{ et }V_2=\frac{2}{3}S_1+\frac{1}{3}S_2$$
Vous devriez voir les traces des deux instruments qui diffèrent par leur structure.*

#### **Visualisation des données initiales**

In [ ]:
Fe=44100
#V1="Mix11.wav"
a=read(V1)
Audio(a[1], rate=Fe)

In [ ]:
Fe=44100
#V2="Mix21.wav"
b=read(V2)
Audio(b[1], rate=Fe)

In [ ]:
plt.figure()
plt.plot(a[1])
plt.show()
plt.plot(b[1])

Nous pouvons constater ici que les deux signaux émettant des deux voix ne sont pas les mêmes. Nous observons une différence notable au niveau de l'intensité des deux voix, cela met en exergue un mélange différent des deux sons sources.

De surcroît, en affichant séparement les signaux sonores des deux voix, nous remarquons deux formes qui se distinguent : d'un côté des pics d'intensité qui correspondraient à la batterie, de l'autre une forme plus étendue en temps pour la guitare. Nous nous attendons à retrouver chaque forme pour chaque spectre.

#### **Affichage des deux TFCT**

In [ ]:
def TFCourtTerme(Son,N,rec):
    H=np.hanning(N)
    NS=len(Son)
    Nf=math.floor(NS*rec/N)-(rec-1)
    TF = np.zeros((N,Nf),dtype='complex')
    for k in range (1,Nf):
        TF[:,k] = np.copy(Son[(k-1)*math.floor(N/rec):(k-1)*math.floor(N/rec)+N]*H) 
        TF[:,k] = fftpack.fft(TF[:,k]) 
    return TF

In [ ]:
son1 = a[1]
TF1_0 = TFCourtTerme(son1,1024,8)

plt.figure(0,figsize=(20, 20))
plt.imshow(np.abs(TF1_0[:500,:]),vmin=10,vmax=200000)

In [ ]:
son2 = b[1]
TF2_0 = TFCourtTerme(son2,1024,8)

plt.figure(0,figsize=(20, 20))
plt.imshow(np.abs(TF2_0[:500,:]),vmin=10,vmax=200000)

Commentaires : 
Nous nous apercevons bien, en observant la TFCT des deux signaux, que les sons ne sont pas les mêmes. Au vu des caractéristiques des deux instruments, nous devinons les effets de ces derniers sur la TFCT. 
La trace de la guitare correspond à un trait allongé dans le temps, alors que celle de la batterie s'allonge sur les fréquences, ceci étant expliqué par la brièveté du son de la batterie. 
Nous pouvons dire que les traces qui s'étalent sur les fréquences sont plus fortes sur le deuxième spectrogramme comme le laissait entendre la comparaison des deux signaux sonores. En revanche, le son de la guitare est plus intense sur la voix 1, ce que l'on observe également sur le spectrogramme. Finalement, les deux TFCT confirment les hypothèses sur les mélanges des deux instruments.

#### **Décomposition du son**

*Proposer un programme Separation2Instru qui prend en entrée deux Transformées de Fourier à court terme (TFCT) et un seuil $T$ et qui renvoie 4 TFCT, chacune associée à une source et à un instrument. Le détail de la procédure est décrite dans le poly et a été expliqué en cours. On rappelle, qu'on effectue la séparation uniquement à partir du module de la TFCT. On reconstruit ensuite les TFCT en utilisant les phases des voix 1 et 2.*

Pour attribuer chaque coefficient au bon spectre, nous calculons le rapport d'intensité en chaque point : si celui-ci est supérieur ou égal au seuil fixé, nous le stockons dans les premiers spectres correspondant associés à ceux de la guitare, sinon nous stockons sa valeur dans ceux de la batterie.

In [ ]:
def Separation2Intru(TF1_0,TF2_0,T): 
    n,m = np.shape(TF1_0)
    
    TF1G = np.zeros((n,m),dtype='complex')
    TF1B = np.zeros((n,m),dtype='complex')
    TF2G = np.zeros((n,m),dtype='complex')
    TF2B = np.zeros((n,m),dtype='complex')
    
    for i in range (n):
        for j in range(m):
            if np.abs(TF1_0[i,j])/np.abs(TF2_0[i,j]) >= T :
                TF1G[i,j] = TF1_0[i,j]               
                TF2G[i,j] = TF2_0[i,j]
            else :
                TF1B[i,j] = TF1_0[i,j]             
                TF2B[i,j] = TF2_0[i,j]
    return TF1G, TF1B, TF2G, TF2B 

*Quel seuil peut-on proposer ici ? Proposer une formule générale en fonction des coefficients du mélange.*

Nous choisissons ici comme seuil la moyenne géométrique $T = \sqrt{R_1 R_2}$, cette dernière moins sensible aux grandes valeurs que la moyenne arithmétique.

In [ ]:
R1 = (1/3)/(2/3)
R2 = (2/3)/(1/3)
T = np.sqrt(R1*R2)
print(T)

Maintenant que nous avons l'algorithme de séparation, nous construisons les spectres correspondant à nos deux instruments pour chaque voix.

In [ ]:
TF1G, TF1B, TF2G, TF2B  = Separation2Intru(TF1_0,TF2_0,T)

In [ ]:
plt.figure(0,figsize=(20, 20))
plt.imshow(np.abs(TF1B[:500,:]),vmin=10,vmax=200000)

In [ ]:
plt.figure(0,figsize=(20, 20))
plt.imshow(np.abs(TF1G[:500,:]),vmin=10,vmax=200000)

En affichant deux des TFCT reconstituées après séparation, nous nous rendons compte que nous avons bien séparé le son émis par les deux instruments. Sur chacune, nous observons la trace laissée par un seul instrument.

#### **Reconstitution des sons séparés**

*Reconstruire ensuite à l'aide du programme RecSon, chacune des voix et écouter chacun des 4 sons produits. 
Commentez le résultat.*

In [ ]:
def RecSon(TF,rec):
    N=np.shape(TF)[0]
    Nf=np.shape(TF)[1]
    H=np.hanning(N)
    Son=np.zeros( math.floor((Nf+7)*N/rec), dtype='complex')
    for k in range (1,Nf):
        d = (k-1)*math.floor(N/rec)
        f = d+N
        Son[d:f] += np.real(fftpack.ifft(TF[:,k])*H)
    return Son

In [ ]:
#Son original
z = RecSon(TFCourtTerme(son1,1024,8),8)
Audio(z,rate=Fe)

In [ ]:
#Son Batterie Voix 1 
a = RecSon(TF1B,8)
Audio(a, rate=Fe)

In [ ]:
plt.plot(a)
plt.show()

In [ ]:
#Son Guitare Voix 1 
b = RecSon(TF1G,8)
Audio(b, rate=Fe)

In [ ]:
plt.plot(b)
plt.show()

In [ ]:
#Son Batterie Voix 2 
c = RecSon(TF2B,8)
Audio(c, rate=Fe)

In [ ]:
plt.plot(c)
plt.show()

In [ ]:
#Son Guitare Voix 2 
d = RecSon(TF2G,8)
Audio(d, rate=Fe)

In [ ]:
plt.plot(d)
plt.show()

Finalement, nous retrouvons pour chaque voix, le son d'un seul des deux instruments, cela se retrouve également sur le signal sonore : nous obtenons les formes caractéristiques de chaque instrument vues précédemment.




## Séparation de trois instruments

#### **Visualisation des données intiales**

*Charger les nouvelles voix 1 et 2 à partir des fichiers Mix12.wav et Mix22.wav.
    On fait l'hypothèse que chacune des voix est maintenant un mélange instantané de trois instruments 
$$SV_1= \frac{2}{5}S_1 + \frac{1}{5}S_2 + \frac{4}{5}S_3\text{ et }V_2= \frac{3}{5}S_1 + \frac{4}{5}S_2 + \frac{1}{5}S_3$$*


In [ ]:
Fe=44100
#V3="Mix12.wav"
a=read(V3)
Audio(a[1], rate=Fe)

In [ ]:
Fe=44100
#V4="Mix22.wav"
b=read(V4)
Audio(b[1], rate=Fe)

*Afficher les spectrogrammes des deux voix et visualiser les traces des trois instruments.* 

In [ ]:
plt.figure()
plt.plot(a[1])
plt.show()
plt.plot(b[1])

Ici encore, nous observons que le signal est différent selon la voix, notamment en terme d'intensité. Cela semble mettre encore une fois en évidence les différents mélange du son. En ce qui concerne les formes caractéristiques des instruments, nous reconnaissons la batterie. En revanche, il est plus compliqué de discerner le piano et la guitare. Toutefois, par l'expérience précédente, nous connaissons le schéma du son de la guitare. Nous pouvons donc deviner celui du piano (plus particulèrement pour la deuxième voix).

#### **Affichage des deux TFCT**

In [ ]:
son1 = a[1]
TF1_0 = TFCourtTerme(son1,1024,8)

plt.figure(0,figsize=(20, 20))
plt.imshow(np.abs(TF1_0[:500,:]),vmin=10,vmax=200000)

In [ ]:
son2 = b[1]
TF2_0 = TFCourtTerme(son2,1024,8)

plt.figure(0,figsize=(20, 20))
plt.imshow(np.abs(TF2_0[:500,:]),vmin=10,vmax=200000)

Nous constatons encore la trace allongée (verticalement) de la batterie.

 Nous pouvons reconnaître la trace de la guitare (les traces jaunes vertes) par la séparation précédente, qui est plus marquée dans la première voix. 
 
 Etant donné nos suppositions sur la batterie et la guitare, nous en déduisons la trace du piano, allongée dans le temps. Cela semble cohérent avec le type de son qu'émet cet instrument. Ce son "fluide" semble plus marqué dans le deuxième spectrogramme. 

#### **Reconstitution des signaux décomposés**

*Proposer un programme de séparation Separation3Instru qui prend en entrée deux TFCT, un vecteur $T$ seuil à 2 composantes et qui renvoie 6 TFCT associées aux deux voix de chacun des trois instruments.*

Dans cet algorithme, nous calculons le rapport d'intensité et nous le comparons aux valeurs seuil : si cette valeur est inférieure à $T_1$ nous associons les coefficients des deux spectres à ceux du piano, si c'est supérieur à $T_2$ nous les associons à ceux de la guitare, sinon à ceux de la batterie.



In [ ]:
def Separation3Intru(TF1_0,TF2_0,T1,T2): 
    n,m = np.shape(TF1_0)
    
    TF1G = np.zeros((n,m),dtype='complex')
    TF1B = np.zeros((n,m),dtype='complex')
    TF1S = np.zeros((n,m),dtype='complex')
    TF2G = np.zeros((n,m),dtype='complex')
    TF2B = np.zeros((n,m),dtype='complex')
    TF2S = np.zeros((n,m),dtype='complex')
    
    for i in range (n):
        for j in range(m):
            num = np.abs(TF1_0[i,j])
            denum = np.abs(TF2_0[i,j]) 
            if num < T1 * denum :
                TF1S[i,j] = TF1_0[i,j]               
                TF2S[i,j] = TF2_0[i,j]
            elif num > T2 * denum :
                TF1G[i,j] = TF1_0[i,j]               
                TF2G[i,j] = TF2_0[i,j]
            else :
                TF1B[i,j] = TF1_0[i,j]             
                TF2B[i,j] = TF2_0[i,j]
    return TF1G, TF1B, TF2G, TF2B , TF1S, TF2S

*Ecrire un programme CalculSeuil qui prend en entrée 3 rapports d'intensité et qui renvoie deux seuils associés.*

Pour calculer, le seuil, ici, nous calculons les rapport d'intensité $R_1$,$R_2$ et $R_3$. Puis, en les ordonnant, nous calculons les moyennes géométriques $T_1$ et $T_2$ selon le même principe que précédemment : $T_1 = \sqrt{R_1 R_2}, T_2 = \sqrt{R_2 R_3}$ en supposant que $R_1 < R_2 < R_3$.

In [ ]:
def CalculSeuil(a11,a12,a13,a21,a22,a23):
    R = np.array([a11/a21,a12/a22,a13/a23])
    R = np.sort(R)
    T1 = np.sqrt(R[0]*R[1])
    T2 = np.sqrt(R[1]*R[2])
    return T1,T2

In [ ]:
T1, T2 = CalculSeuil(2./5,1./5,4./5,3./5,4./5,1./5)
print(T1, T2)

*Tester le programme en utilisant les rapports d'intensités calculés à partir des coefficients de mélange, et commenter.*

Maintenant que nous avons adapté l'algorithme de séparation pour trois instruments, nous construisons les six spectrogrammes.

In [ ]:
TF1G, TF1B, TF2G, TF2B, TF1S, TF2S  = Separation3Intru(TF1_0,TF2_0,T1,T2)

#### **Reconsitution des sons séparés**

In [ ]:
Fe=44100
#V2="Mix22.wav"
b=read(V4)
Audio(b[1], rate=Fe)

In [ ]:
#Son Batterie Voix 1 
a = RecSon(TF1B,8)
Audio(a, rate=Fe)

In [ ]:
plt.plot(a)
plt.show()

In [ ]:
#Son Batterie Voix 2 
b = RecSon(TF2B,8)
Audio(b, rate=Fe)

In [ ]:
#Son Guitare Voix 1 
c = RecSon(TF1G,8)
Audio(c, rate=Fe)

In [ ]:
plt.plot(c)
plt.show()

In [ ]:
#Son Guitare Voix 2
d = RecSon(TF2G,8)
Audio(d, rate=Fe)

In [ ]:
#Son Piano Voix 1 
e = RecSon(TF1S,8)
Audio(e, rate=Fe)

In [ ]:
plt.plot(e)
plt.show()

In [ ]:
#Son Piano Voix 2
f = RecSon(TF2S,8)
Audio(f, rate=Fe)

Commentaires : 
On peut voir sur les graphes que les sons ont bien été séparés, ce qui est également confirmé en écoutant les bandes sonores. En revanche, la batterie et le piano se recouvrent un peu. Ainsi, nous entendons du piano dans la bande son dédiée à la batterie, enlevant de la qualité à ce dernier ainsi qu'à celui dédié au piano.

# **Séparation sans connaissance des coefficients de mélange**

## Estimation des coefficients de mélange

On suppose dans cette partie qu'on veut effectuer la séparation précédente, sans connaitre les coefficienst de mélange. On parle de séparation aveugle. On va donc chercher à estimer ces coefficients de mélange, ou plus précisément des rapports d'intensité des deux voix pour chacun des instruments. L'idée générale est simple : on va calculer un histogramme des rapports d'intensité des modules des TFCT et sélectionner les 3 valeurs les plus réprésentées dans l'histogramme, c'est à dire les troix maxima locaux de l'histogramme. Dans le détail on procéder de la manière suivante :

1) On construit un tableau ModuleCarre qui va contenir la somme des carrés des modules des TFCT des deux voix.

2) On seuille ce tableau de manière à ne conserver que les N= 10% de coefficients les plus importants du tableau.

3) On construit une matrice $2\times N$ qui contient les pairs d'intensité associées des TFCT des deux voix.

4) On affiche sur un diagramme 2D les points obtnus (on ne les relie pas, on affiche par exemple des croix).

Les points obtenus doivent se répartir approximativement selon des nuages dirigés le long de droites dont les coefficients directeurs sont les rapports d'intensité.

5) On calcule un vecteur $R$ de rapports d'intensité (de longueur N).

6) On affiche un histogramme de $R$ et un histogramme de $U=\frac{R}{1+R}$. Vous devriez observer que les maxima de l'histogramme de $U$ sont plus marqués. 

7) Ecrire un programme qui estime les 3 valeurs des maximas de l'histogramme de U et donc les rapports $R_1$, $R_2$ et $R_3$ associés aux trois isntruments.

8) A l'aide du programme précédent, calculer les seuils optimaux et effectuer la séparation.

9) Commenter.

#### **Sélection des 10% coefficients les plus importants**

Nous calculons ici la somme des carrés des modules des TFCT des deux voix, le résultat étant placé dans la variable ModuleCarre. Nous sélectionnons les couples $(i,j)$ tel que $ModuleCarre_{i,j}$ fasse parti des $10$% plus grands  coefficients de la matrice. En sortie, nous fournissons la valeur de la TFCT en ces couples $(i,j)$. Une fois cette sortie obtenue, nous affichons la TFCT de la voix 1 en fonction de celle de la voix 2.

In [ ]:
def intensite (TF1,TF2):
    n,m = np.shape(TF1)
    
    ModuleCarre = np.abs(TF1)**2 + np.abs(TF2)**2 
    ModuleCarre = ModuleCarre.reshape((1,n*m))
    index = np.argsort(-ModuleCarre)[0]
    N = int(0.1*len(index))
    index = index[:N]
    PInt = np.zeros((2,N))
    for k in range (N):
        i = index[k]//m
        j = index[k]%m                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
        PInt[0,k] = np.abs(TF1[i,j])
        PInt[1,k] = np.abs(TF2[i,j])
    return PInt
    
plt.figure()
PInt = intensite(TF1_0,TF2_0)
plt.plot(PInt[0,:],PInt[1,:],'x')
plt.title("TFCT1 en fonction de TFCT2 pour certains coefficients")

Sur ce graphe, correspondant à la TFCT1 en fonction de la TFCT2 pour les 10% coefficients les plus élevés de la somme de leurs modules carrés, nous pouvons constater que le nuage de points se répartit le long de deux droites. Leur coefficient directeur correspond aux rapports d'intensité qui nous intéressent.
Ici, il nous semble difficile de les retrouver par une régression linéaire. C'est pourquoi, nous allons essayer de les retrouver au travers d'un histogramme de $\frac{TFCT1}{TFCT2}$.   

#### **Estimation des rapports $R$**

Nous affichons ici l'histogramme correspondant à $\frac{TFCT_1(i,j)}{TFCT_2(i,j)}$, où $TFCT_1$ est la TFCT de la voix 1, et $TFCT_2$ de la voix 2, et $(i,j)$ correspond aux couples sélectionnés précédemment.

In [ ]:
R = PInt[0,:]/PInt[1,:]
plt.figure(figsize=(15,4))
plt.subplot(1,2,1)
plt.hist(R,bins=len(R)//1000)
plt.title("Histogramme de R")

plt.subplot(1,2,2)
plt.hist(R[R<25],bins=len(R[R<25])//1000)
plt.title("Histogramme de R sur [0,25]")
plt.show()

Ici, nous constatons d'une part des valeurs plutôt aberrantes (l'échelle des abscisses est supérieure à 100). De plus, il est difficile d'observer les trois pics correspondant à $R_1, R_2, R_3$.

Pour pallier cela, nous nous proposons de tracer l'histogramme de $U$, où $U = \frac{R}{1+R}$. Cela permet de concentrer davantage l'information pour retrouver plus facilement les maxima. De plus, la fonction f : R $\longmapsto$ U étant inversible, nous saurons retrouver R.

In [ ]:
U = R/(R+1)
plt.hist(U, bins=len(U)//1000)
plt.title("Histogramme de U")
plt.show()

Nous observon bien ici les trois maximas. Il ne reste plus qu'à obtenir les $argmax$. 

La fonction suivante se charge de réaliser cela. Elle se contente de sélectionner les points représentants des $max$,c'est-à-dire tel que $U_{k-1} < U_{k}$ et $U_{k} < U_{k+1}$. Parmi ceux-là, elle ne considère que les $3$ premiers et retourne leurs positions dans le tableau constituant l'histogramme. Grâce à cette dernière, nous pouvons trouver les bornes utilisées pour faire l'histogramme. L'$argmax$ correspond alors à la moyenne des deux bornes (par choix arbitraire).   

In [ ]:
def histmax(hist):
    max_pot = []
    max_hist = []
    for i in range (1,len(hist)-1):
        if hist[i] - hist[i-1] > 0 and hist[i+1] - hist[i] < 0 :
            max_pot += [i]
    index = np.argsort(-hist[max_pot])[:3]
    for i in range (3):
        max_hist += [max_pot[index[i]]] 
    return max_hist


In [ ]:
hist, bin_edges = np.histogram(U,len(U)//1000)
plt.plot(hist)

index = histmax(hist)
print(index)
Umax = np.zeros(len(index))
for i in range (len(index)):
    k = index[i]
    Umax[i] = (bin_edges[k]+bin_edges[k+1])/2
    

Par ce test, nous vérifions les argmax de U.

In [ ]:
Rmax = Umax/(1-Umax)
print(Umax)
print(Rmax)

Notons que ce résultat est très variable en fonction des intervalles choisis pour l'histogrammme. En effet, si nous prenons des intervalles trop grands, nous perdons en précision. Si nous les prenons trop petits, nous pouvons confondre et nous tromper sur les maxima locaux. En effet, nous constatons que l'amplitude du deuxième pic est très forte, alors que le premier pic est peu visible. Si nous découpons en de plus petits intervalles, nous pouvons prendre deux fois le même $max$.

## Application

Maintenant que nous avons retrouvé nos rapports d'intensité, nous pouvons séparer les trois instruments. Toutefois, nous devons encore adapter la fonction qui calcule les seuils, ne prenant plus en argument les coefficients du mélange mais ces rapports d'intensité.



In [ ]:
def CalculSeuil(R):
    R = np.sort(R)
    T1 = np.sqrt(R[0]*R[1])
    T2 = np.sqrt(R[1]*R[2])
    return T1,T2

In [ ]:
T1, T2 = CalculSeuil(Rmax)
print(T1, T2)
TF1G, TF1B, TF2G, TF2B, TF1S, TF2S  = Separation3Intru(TF1_0,TF2_0,T1,T2)

In [ ]:
#Son Batterie Voix 1 
a = RecSon(TF1B,8)
Audio(a, rate=Fe)

In [ ]:
plt.plot(a)
plt.show()

In [ ]:
#Son Batterie Voix 2
b = RecSon(TF2B,8)
Audio(b, rate=Fe)

In [ ]:
#Son Guitare Voix 1 
c = RecSon(TF1G,8)
Audio(c, rate=Fe)

In [ ]:
plt.plot(c)
plt.show()

In [ ]:
#Son Guitare Voix 2
d = RecSon(TF2G,8)
Audio(d, rate=Fe)

In [ ]:
#Son Piano Voix 1
e = RecSon(TF1S,8)
Audio(e, rate=Fe)

In [ ]:
plt.plot(e)
plt.show()

In [ ]:
#Son Piano Voix 2
f = RecSon(TF2S,8)
Audio(f, rate=Fe)

Commentaires : 
Nous avons séparé les différents instruments en retrouvant expérimentalement les rapports d'intensité. Nous sommes globalement contents des résultats puisque nous avons à peu de choses près les mêmes sorties que lorsque nous connaissions le mélange. 